# AI 201 Programming Assignment 1
## A* Algorithm Implementation

Submitted by: 
Jan Lendl R. Uy, 2019-00312

### Notebook Prerequisites
Install the following Python modules, if not yet installed:
1. numpy

In [38]:
import copy

In [39]:
# CONSTANTS
INPUT_FILE_PATH = "astar_in.txt"
MAX_ALGORITHM_ITERATIONS = 2

In [40]:
def read_file(path):
    
    with open(path, "r") as file:
        lines = file.readlines()

    # Initialize lists to store the start state and goal state
    start_state = []
    goal_state = []

    # Initialize a flag to keep track if following tiles belong to the start
    # or the goal state
    reading_part = None

    for line in lines:
        line = line.strip() # Remove leading/trailing whitespace
        if line == "start":
            reading_part = "start"
        elif line == "goal":
            reading_part = "goal"
        elif line and reading_part:
            # Convert line into a list, handling '*' and converting numbers to integers
            line_list = [int(x) if x.isdigit() else x for x in line.split()]
            if reading_part == "start":
                start_state.append(line_list)
            elif reading_part == "goal":
                goal_state.append(line_list)

    return start_state, goal_state

start, goal = read_file(INPUT_FILE_PATH)
print(f"Start State: {start}")
print(f"Goal State: {goal}")

Start State: [['*', 1, 3], [8, 2, 4], [7, 6, 5]]
Goal State: [[1, 2, 3], [8, '*', 4], [7, 6, 5]]


In [41]:
class PuzzleBoard:
    
    def __init__(self, board_as_list):
        self.board = board_as_list
        self.blank_tile_coords = self.get_blank_tile_coords()
        self.valid_movements = self.get_valid_movements()   
       
    # Print override
    # Formats the Puzzle printing as a 3x3 grid-like structure 
    def __str__(self):
        board_in_str = ""
        for row in self.board:
            # print(f"row = {row}")
            for tile in row:
                # print(f"tile = {tile}")
                board_in_str += f"| {tile} "
            board_in_str += "|\n"

        return board_in_str
    
    # Equality override
    # Checks if two boards have exactly the same elements in 
    # the same location 
    def __eq__(self, other_puzzle):
        other_board = other_puzzle.board
        for i in range(len(other_board)):
            for j in range(len(other_board[i])):
                if other_board[i][j] != self.board[i][j]:
                    return False
        return True
    
    def add_coordinates(self, coords_1, coords_2):
        return tuple(map(lambda a, b: a + b, coords_1, coords_2))    
    
    def swap(self, coords_orig, coords_new):
        x, y = coords_orig
        new_x, new_y = coords_new
        board_copy = copy.deepcopy(self.board)
        board_copy[x][y], board_copy[new_x][new_y] = board_copy[new_x][new_y], board_copy[x][y]
        return board_copy
        
    def get_valid_movements(self):
        # Define possible movements: up, down, left, right
        movements = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}
        
        # Define a dictionary to store valid movements and the corresponding 
        # indices of swaps
        self.valid_movements = {}
        
        # Get valid movements
        for movement in movements:
            check_valid_movement = self.add_coordinates(self.blank_tile_coords, movements[movement])
            x_moved, y_moved = check_valid_movement
            # new_coords.append(check_valid_movement)
            if x_moved < 0 or y_moved < 0:
                continue
            self.valid_movements[movement] = check_valid_movement
            
        return self.valid_movements     
    
    def get_blank_tile_coords(self):
        blank_tile = None
        for row in self.board:
            for tile in row:
                if tile == "*":
                    blank_tile = (self.board.index(row), row.index(tile))
        return blank_tile

starting_puzzle = PuzzleBoard(start)
goal_puzzle = PuzzleBoard(goal)
print(f"Start State: \n{starting_puzzle}")
print(f"Goal State: \n{goal_puzzle}")

Start State: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Goal State: 
| 1 | 2 | 3 |
| 8 | * | 4 |
| 7 | 6 | 5 |



In [42]:
def a_star(start, goal, heuristic=None):
    
    open = []
    closed = set()
    
    current_node = start
                    
    # print(f"Blank tile coordinates: {start.blank_tile_coords}")
    print(f"Start node: \n{start}")
    open.append(start)
    ctr = 0

    while ctr < MAX_ALGORITHM_ITERATIONS:
            
        # If open is empty, return failure and end search
        # Else, continue search
        if len(open) < 1:
            print("No solution found.")
            break
        
        # If current node is the goal node, return success
        # End search
        if (current_node == goal):
            print(f"Goal achieved! \n{current_node}")
            break
        
        # Expand current node then add these nodes to the list of
        # opened nodes
        successors = get_successors(current_node)
        
        # Print opened nodes for checking
        print(f"Step {ctr+1}: Opened Nodes:\n")
        for successor in successors:
            print(f"Successor node: \n{successor}")
            
        open += successors
        ctr += 1
    
def get_successors(start):
    
    successors = []     
    
    # Move the tile according to the specified direction
    for movement in start.valid_movements:
        successor = start.swap(start.blank_tile_coords, start.valid_movements[movement])
        successor_puzzle = PuzzleBoard(successor)
        successors.append(successor_puzzle)
        
    return successors

path = a_star(starting_puzzle, goal_puzzle)
if path:
    for state in path:
        print(state)
else:
    print("No solution found.")

Start node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Step 1: Opened Nodes:

Successor node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |

Successor node: 
| 1 | * | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Step 2: Opened Nodes:

Successor node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |

Successor node: 
| 1 | * | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

No solution found.


In [43]:
# import heapq

# def a_star(start, goal, heuristic=None):
#     # Helper function to reconstruct the path from the goal to the start
#     def reconstruct_path(came_from, current):
#         path = [current]
#         while current in came_from:
#             current = came_from[current]
#             path.insert(0, current)
#         return path

#     # Start with just the start node in the open list (heap queue)
#     open_list = []
#     heapq.heappush(open_list, (number_of_wrong_tiles(start, goal), start))
#     came_from = {start: None}
#     g_score = {start: 0}  # Cost from start to the current node
#     f_score = {start: number_of_wrong_tiles(start, goal)}  # Estimated cost from start to goal

#     # Set of nodes already evaluated
#     closed_set = set()

#     while open_list:
#         _, current = heapq.heappop(open_list)

#         # If the goal is reached, reconstruct and return the path
#         if current == goal:
#             return reconstruct_path(came_from, current)

#         closed_set.add(current)

#         # Generate children (successors)
#         for neighbor in get_successors(current):
#             tentative_g_score = g_score[current] + 1  # Assuming each step cost is 1

#             if neighbor in closed_set and tentative_g_score >= g_score.get(neighbor, float('inf')):
#                 continue  # This is not a better path

#             if tentative_g_score < g_score.get(neighbor, float('inf')):
#                 # This path is the best so far, record it
#                 came_from[neighbor] = current
#                 g_score[neighbor] = tentative_g_score
#                 f_score[neighbor] = tentative_g_score + number_of_wrong_tiles(neighbor, goal)
#                 if neighbor not in [i[1] for i in open_list]:
#                     heapq.heappush(open_list, (f_score[neighbor], neighbor))

#     # Open list is empty but goal was never reached
#     return None

# # Heuristic function: Number of wrong tiles
# def number_of_wrong_tiles(current_state, goal_state):
#     wrong_counter = 0
#     for i in range(len(start)):
#         for j in range(len(start[i])):
#             if current_state[i][j] != goal_state[i][j]:
#                 wrong_counter += 1
    
#     return wrong_counter

# # Successor function to generate children of a node
# def get_successors(state):
#     # First, find the position of the empty space denoted by '*'
#     empty_pos = [(ix, iy) for ix, row in enumerate(state) for iy, i in enumerate(row) if i == '*'][0]
#     x, y = empty_pos

#     # Define possible movements: up, down, left, right
#     movements = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}

#     # List to store successor states
#     successors = []

#     # Generate all possible moves within the bounds of the puzzle
#     for move in movements.values():
#         new_x, new_y = x + move[0], y + move[1]

#         # Check if the new position is within the bounds of the puzzle
#         if 0 <= new_x < len(state) and 0 <= new_y < len(state[0]):
#             # Make a deep copy of the current state to create a new state
#             new_state = [row[:] for row in state]
#             # Swap the empty space with the adjacent tile
#             new_state[x][y], new_state[new_x][new_y] = new_state[new_x][new_y], new_state[x][y]
#             # Add the new state to the list of successors
#             successors.append(new_state)

#     return successors


# path = a_star(start, goal, "number_of_wrong_tiles")
# if path:
#     for state in path:
#         print(state)
# else:
#     print("No solution found.")
